In [4]:
import angr

In [5]:
# Load the project
b = angr.Project("C:/Users/jetti/github/angr-examples/examples/fauxware/fauxware", load_options={"auto_load_libs": False})

In [3]:

# Generate a CFG first. In order to generate data dependence graph afterwards, you'll have to:
# - keep all input states by specifying keep_state=True.
# - store memory, register and temporary values accesses by adding the angr.options.refs option set.
# Feel free to provide more parameters (for example, context_sensitivity_level) for CFG
# recovery based on your needs.
cfg = b.analyses.CFGEmulated(keep_state=True,
                             state_add_options=angr.sim_options.refs,
                             context_sensitivity_level=2)

# Generate the control dependence graph
cdg = b.analyses.CDG(cfg)

# Build the data dependence graph. It might take a while. Be patient!
ddg = b.analyses.DDG(cfg)

# See where we wanna go... let's go to the exit() call, which is modeled as a
# SimProcedure.
target_func = cfg.kb.functions.function(name="exit")
# We need the CFGNode instance
target_node = cfg.get_any_node(target_func.addr)

# Let's get a BackwardSlice out of them!
# ``targets`` is a list of objects, where each one is either a CodeLocation
# object, or a tuple of CFGNode instance and a statement ID. Setting statement
# ID to -1 means the very beginning of that CFGNode. A SimProcedure does not
# have any statement, so you should always specify -1 for it.
bs = b.analyses.BackwardSlice(cfg, cdg=cdg, ddg=ddg, targets=[ (target_node, -1) ])

print(bs)

WARNING  | 2023-11-05 06:59:25,653 | angr.storage.memory_mixins.default_filler_mixin | The program is accessing register with an unspecified value. This could indicate unwanted behavior.
WARNING  | 2023-11-05 06:59:25,654 | angr.storage.memory_mixins.default_filler_mixin | angr will cope with this by generating an unconstrained symbolic variable and continuing. You can resolve this by:
WARNING  | 2023-11-05 06:59:25,654 | angr.storage.memory_mixins.default_filler_mixin | 1) setting a value to the initial state
WARNING  | 2023-11-05 06:59:25,655 | angr.storage.memory_mixins.default_filler_mixin | 2) adding the state option ZERO_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to make unknown regions hold null
WARNING  | 2023-11-05 06:59:25,655 | angr.storage.memory_mixins.default_filler_mixin | 3) adding the state option SYMBOL_FILL_UNCONSTRAINED_{MEMORY,REGISTERS}, to suppress these messages.
WARNING  | 2023-11-05 06:59:25,656 | angr.storage.memory_mixins.default_filler_mixin | Filling register 

BackwardSlice (to [(<CFGENode 0x400570[6]>, -1)])


In [9]:
bs.